In [2]:
import torch
import numpy as np

a = torch.arange(9).reshape(3, 3)
# i = 3
torch_ein_out = torch.einsum('ii->i', [a])
torch_org_out = torch.diagonal(a, 0)
# is torch_org_out == torch_ein_out ? True

In [3]:
torch_ein_out

tensor([0, 4, 8])

In [4]:
torch_org_out

tensor([0, 4, 8])

In [6]:
import torch
from einops import rearrange
i_tensor = torch.randn(16, 3, 224, 224)
print(i_tensor.shape)
o_tensor = rearrange(i_tensor, 'n c (m1 p1) (m2 p2) -> n (m1 m2) (p1 p2 c)',p1=16, p2=16)
print(o_tensor.shape)

torch.Size([16, 3, 224, 224])
torch.Size([16, 196, 768])


In [15]:
import torch

i_a = torch.randn(16, 3, 224, 224)
out = torch.einsum('bijk->bjki', i_a)
print(out.shape)

torch.Size([16, 224, 224, 3])


In [16]:
qkv = torch.randn(16, 128, 768*3)

q, k, v = tuple(rearrange(qkv, 'b t (d k h) -> k b h t d ', k=3, h=8))


In [18]:
print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([16, 8, 128, 96])
torch.Size([16, 8, 128, 96])
torch.Size([16, 8, 128, 96])


In [44]:
# resulted shape will be: [batch, heads, tokens, tokens]
scale_factor =0.1
scaled_dot_prod = torch.einsum('b h i d , b h j d -> b h i j', q, k) * scale_factor
print(scaled_dot_prod.shape)

mask =torch.ones(16,8,128,128)
#mask[:,:,125,125]=1

#print(mask)
scaled_dot_prod = scaled_dot_prod.masked_fill(mask.byte(), value=torch.tensor(-1e8))
    
print(scaled_dot_prod[:,:,125,125])

attention = torch.softmax(scaled_dot_prod, dim=-1)
print(attention.shape)
# Step 4. Calc result per batch and per head h
out = torch.einsum('b h i j , b h j d -> b h i d', attention, v)
print(out.shape)
# Step 5. Re-compose: merge heads with dim_head d
out = rearrange(out, "b h t d -> b t (h d)")
print(out.shape)
# Step 6. Apply final linear transformation layer

torch.Size([16, 8, 128, 128])
tensor([[-100000000., -100000000., -100000000., -100000000., -100000000.,
         -100000000., -100000000., -100000000.],
        [-100000000., -100000000., -100000000., -100000000., -100000000.,
         -100000000., -100000000., -100000000.],
        [-100000000., -100000000., -100000000., -100000000., -100000000.,
         -100000000., -100000000., -100000000.],
        [-100000000., -100000000., -100000000., -100000000., -100000000.,
         -100000000., -100000000., -100000000.],
        [-100000000., -100000000., -100000000., -100000000., -100000000.,
         -100000000., -100000000., -100000000.],
        [-100000000., -100000000., -100000000., -100000000., -100000000.,
         -100000000., -100000000., -100000000.],
        [-100000000., -100000000., -100000000., -100000000., -100000000.,
         -100000000., -100000000., -100000000.],
        [-100000000., -100000000., -100000000., -100000000., -100000000.,
         -100000000., -100000000., 

C:\Users\myf\AppData\Local\Temp\ipykernel_19676\2492755303.py:10: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1582.)
  scaled_dot_prod = scaled_dot_prod.masked_fill(mask.byte(), value=torch.tensor(-1e8))


In [43]:
scaled_dot_prod.dtype

torch.float32

In [ ]:
va = torch.randn(5,6)

x = [5,4,3,2,1]
mask = torch.zeros(5,6,dtype=torch.float)
for e_id, src_len in enumerate(x):
    mask[e_id, src_len:] = 1
mask = mask.to(device = 'cpu')
print(mask)
a.data.masked_fill_(mask.byte(),-float('inf'))
print(a)
